In [1]:
import keras
import numpy as np
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras import (
    layers,
    Model,
    Sequential,
    optimizers
)

from artificial_gps.settings import (
    INPUT_SEQUENCE_LEN,
    INPUT_DATA_COLUMNS,
    OUTPUT_DATA_COLUMNS
)
print(1)

In [8]:
force_cpu_run = True
if force_cpu_run:
    tf.config.set_visible_devices([], 'GPU')

print(INPUT_DATA_COLUMNS)
print(OUTPUT_DATA_COLUMNS)
def create_model() -> Model:
    """
    Creates dl model
    :param hp: Hyper parameters (Initialized by a tuning algorithm)
    :return:
    """
    model = Sequential()
    model.add(layers.Input(len(INPUT_DATA_COLUMNS)))
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(len(OUTPUT_DATA_COLUMNS)))

    optimizer = optimizers.Adam(learning_rate=0.01)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    return model

In [9]:
from artificial_gps.data import load_preprocessed_dataset
train_x, train_y, dev_x, dev_y, test_x, test_y = load_preprocessed_dataset()

In [5]:
model = create_model()

history = model.fit(train_x,
                    train_y,
                    epochs=30,
                    batch_size=64,
                    validation_data=(dev_x, dev_y))

In [6]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"])

plt.plot(history.history["val_loss"])

In [8]:
test_x[:20]

array([[5.02179467e-01, 5.03817945e-01, 5.12156865e-01, 4.95403350e-01,
        4.93496078e-01, 4.95150901e-01, 5.04034981e-01, 4.95235101e-01,
        6.19238326e-01, 2.90379315e-01, 5.73044786e-01, 3.93757074e-01,
        4.67919185e-01, 5.86301341e-01, 5.75165075e-01, 9.91752394e-01,
        5.75757576e-01, 8.04191995e-01, 1.94262908e-01, 0.00000000e+00,
        5.99998537e-01, 7.26132262e-01, 4.41023721e-01, 7.26132262e-01,
        5.99998537e-01, 8.31943562e-01, 6.92130206e-01, 3.07869818e-01,
        8.33490358e-01, 6.94706210e-01, 3.05293796e-01, 8.32933726e-01,
        6.93778593e-01, 6.93778539e-01, 8.32495273e-01, 6.93048411e-01,
        6.93048394e-01, 3.51852408e-01],
       [5.01121162e-01, 5.03314908e-01, 5.12146252e-01, 4.95772131e-01,
        4.94856585e-01, 4.95199155e-01, 5.11442858e-01, 4.90961203e-01,
        6.52383600e-01, 3.17283468e-01, 5.38090190e-01, 1.51969704e-01,
        4.24357224e-01, 4.75708557e-01, 6.60673689e-02, 6.98810124e-01,
        4.54545455e-01,

In [17]:
model.predict(train_x[180:200])

array([[0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ],
       [0.01807293, 0.6583927 , 0.6423819 ]], dtype=float32)

In [18]:
test_y[180:200]

array([[0.01367268, 0.65565527, 0.64530118],
       [0.02140996, 0.66254651, 0.6416053 ],
       [0.01554934, 0.65341733, 0.64167284],
       [0.01514493, 0.65748661, 0.64186629],
       [0.01108591, 0.65926129, 0.64104388],
       [0.01603092, 0.65576585, 0.64329908],
       [0.01439923, 0.65813575, 0.64182823],
       [0.01259884, 0.66201449, 0.64272588],
       [0.01900475, 0.65716926, 0.64342539],
       [0.01803082, 0.65813575, 0.64182823],
       [0.01840263, 0.65788673, 0.64243192],
       [0.01820192, 0.65855495, 0.64202377],
       [0.01832067, 0.65801793, 0.64188788],
       [0.01368135, 0.65829804, 0.6405231 ],
       [0.01663394, 0.65386474, 0.63667867],
       [0.01844779, 0.65768467, 0.6427855 ],
       [0.02693586, 0.66230234, 0.64096213],
       [0.01519339, 0.66095331, 0.6398844 ],
       [0.01622415, 0.65629418, 0.64246724],
       [0.01397209, 0.6550782 , 0.64415979]])

In [1]:
from artificial_gps.data import load_dataset
train_x, train_y, dev_x, dev_y, test_x, test_y = load_dataset()
import matplotlib.pyplot as plt

plt.hist(train_y[:200, 0], 10000)